# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training.

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.

Algorithms:

(1) MultinominalNB

(2) SVM

(3) KNN

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison

(4) F-1 score

In [1]:
# Write your code here

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import files
uploaded = files.upload()

Saving stsa-test.txt to stsa-test.txt
Saving stsa-train.txt to stsa-train.txt


In [3]:
# Opening and reading the contents of the "stsa-train.txt" file
with open("stsa-train.txt") as txt_file:
    lines = [line.rstrip('\n') for line in txt_file]

# Separating sentiment labels and text content from each line in the file
sentiments = []
texts = []

for line in lines:
    # Extracting sentiment label and text content from each line
    sentiment = line[0]
    text_content = line[1:]

    # Appending sentiment label and text content to their respective lists
    sentiments.append(sentiment)
    texts.append(text_content)

# Creating a DataFrame from the lists of sentiment labels and text content
dataset = pd.DataFrame(list(zip(sentiments, texts)), columns=['Sentiment', 'Text'])
dataset.head()

# Printing the first few rows of the dataset for inspection
print(dataset.head())


  Sentiment                                               Text
0         1   a stirring , funny and finally transporting r...
1         0   apparently reassembled from the cutting-room ...
2         0   they presume their audience wo n't sit still ...
3         1   this is a visually stunning rumination on lov...
4         1   jonathan parker 's bartleby should have been ...


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

# Downloading NLTK resources
nltk.download()

# Initializing lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence_text = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence_text = sentence_text.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence_text = re.sub(cleaner_regex, '', sentence_text)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence_text)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in word_tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a processed text
    return " ".join(filtered_words)

# Applying the preprocessing function to the 'Text' column and creating a new 'processedText' column
dataset['processedText'] = dataset['Text'].map(lambda s: preprocess_text(s))

# Displaying the first few rows of the dataset with the processed text
dataset.head()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


,Sentiment,Text,processedText
0,1,"a stirring , funny and finally transporting r...",stirring funny finally transporting imagining ...
1,0,apparently reassembled from the cutting-room ...,apparently reassembled cutting room floor give...
2,0,they presume their audience wo n't sit still ...,presume audience sit still sociology lesson ho...
3,1,this is a visually stunning rumination on lov...,visually stunning rumination love memory histo...
4,1,jonathan parker 's bartleby should have been ...,jonathan parker bartleby end modern office ano...


In [7]:
# Opening and reading the contents of the "stsa-test.txt" file
with open("stsa-test.txt") as test_txt_file:
    lines_test = [line.rstrip('\n') for line in test_txt_file]

# Separating sentiment labels and text content from each line in the test file
sentiments_test = []
texts_test = []

for line_test in lines_test:
    # Extracting sentiment label and text content from each line in the test file
    sentiment_test = line_test[0]
    text_content_test = line_test[1:]

    # Appending sentiment label and text content to their respective lists
    sentiments_test.append(sentiment_test)
    texts_test.append(text_content_test)

# Creating a DataFrame from the lists of sentiment labels and text content for the test dataset
dataset_test = pd.DataFrame(list(zip(sentiments_test, texts_test)), columns=['Sentiment', 'Text'])
dataset_test.head()


,Sentiment,Text
0,0,"no movement , no yuks , not much of anything ."
1,0,"a gob of drivel so sickly sweet , even the ea..."
2,0,"gangs of new york is an unapologetic mess , w..."
3,0,we never really feel involved with the story ...
4,1,this is one of polanski 's best films .


In [8]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

# Download NLTK resources
nltk.download()

# Initializing lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence = sentence.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence = re.sub(cleaner_regex, '', sentence)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a clean text
    return " ".join(filtered_words)

# Applying the preprocessing function to the 'Text' column and creating a new 'cleanText' column for the test dataset
dataset_test['processedText'] = dataset_test['Text'].map(lambda s: preprocess_text(s))

# Displaying the first few rows of the test dataset with the cleaned text
dataset_test.head()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


,Sentiment,Text,processedText
0,0,"no movement , no yuks , not much of anything .",movement yuks much anything
1,0,"a gob of drivel so sickly sweet , even the ea...",gob drivel sickly sweet even eager consumers m...
2,0,"gangs of new york is an unapologetic mess , w...",gangs new york unapologetic mess whose saving ...
3,0,we never really feel involved with the story ...,never really feel involved story ideas remain ...
4,1,this is one of polanski 's best films .,one polanski best films


In [9]:
# Import the TfidfVectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, analyzer='word')

# Transforming the training dataset text into TF-IDF features
train_tfidf_features = tfidf_vectorizer.fit_transform(dataset["processedText"]).toarray()

# Transforming the test dataset text into TF-IDF features
test_tfidf_features = tfidf_vectorizer.transform(dataset_test["processedText"]).toarray()

# Preparing the test dataset features and labels
x_test = test_tfidf_features
y_test = dataset_test["Sentiment"]

# Data partitioning
from sklearn.model_selection import train_test_split

# Splitting the training dataset into training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tfidf_features, dataset["Sentiment"], test_size=0.2, random_state=2)


In [ ]:
# Multinomial Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

# Creating an instance of the Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()

# Training the model on the training data
naive_bayes_model = naive_bayes_classifier.fit(x_train, y_train)

# Making predictions on the validation set
alternative_predictions_validation = naive_bayes_classifier.predict(x_valid)

# Evaluating the performance of the Naive Bayes model on the validation set
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Calculating and printing accuracy
accuracy = accuracy_score(y_valid, alternative_predictions_validation)
print("Validation set accuracy for the Naive Bayes model: {:.2%}".format(accuracy))

# Calculating and printing precision
precision = precision_score(y_valid, alternative_predictions_validation, pos_label='0')
print("Validation set precision for the Naive Bayes model: {:.2%}".format(precision))

# Calculating and printing recall
recall = recall_score(y_valid, alternative_predictions_validation, pos_label='0')
print("Validation set recall for the Naive Bayes model: {:.2%}".format(recall))

# Calculating and printing F1 score
f1 = f1_score(y_valid, alternative_predictions_validation, average='weighted')
print("Validation set F1 score for the Naive Bayes model: {:.2%}".format(f1))

Validation set accuracy for the Naive Bayes model: 77.31%
Validation set precision for the Naive Bayes model: 80.92%
Validation set recall for the Naive Bayes model: 68.98%
Validation set F1 score for the Naive Bayes model: 77.13%


In [ ]:
#Classification Report
from sklearn.metrics import classification_report as clf_report

# Generating a classification report for the Naive Bayes model on the validation set
classification_report_naive_validation = clf_report(y_valid, alternative_predictions_validation)
print("Classification Report for the Naive Bayes model on the validation set:\n", classification_report_naive_validation)


Classification Report for the Naive Bayes model on the validation set:
               precision    recall  f1-score   support

           0       0.81      0.69      0.74       664
           1       0.75      0.85      0.80       720

    accuracy                           0.77      1384
   macro avg       0.78      0.77      0.77      1384
weighted avg       0.78      0.77      0.77      1384



In [ ]:
#Cross-Validation Score
from sklearn.model_selection import cross_val_score as cv_score

# Creating an instance of the Naive Bayes classifier for cross-validation
naive_bayes_classifier_cv = MultinomialNB()

# Calculating 10-fold cross-validation scores for the Naive Bayes model on the training set
naive_accuracies_validation_cv = cv_score(estimator=naive_bayes_classifier_cv, X=x_train, y=y_train, cv=10)

# Printing the mean cross-validation score
print(f"Mean 10-fold cross-validation score for the Naive Bayes model on the training set: {round(naive_accuracies_validation_cv.mean()*100)}%")

Mean 10-fold cross-validation score for the Naive Bayes model on the training set: 77%


In [ ]:
# Making predictions on the test set using the trained Naive Bayes classifier
predictions_test_set = naive_bayes_classifier.predict(x_test)

# Evaluating the performance of the Naive Bayes model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculating and printing accuracy on the test set
test_set_accuracy = accuracy_score(y_test, predictions_test_set)
print("Test set accuracy for the Naive Bayes model: {:.2%}".format(test_set_accuracy))

# Calculating and printing precision on the test set
test_set_precision = precision_score(y_test, predictions_test_set, pos_label='0')
print("Test set precision for the Naive Bayes model: {:.2%}".format(test_set_precision))

# Calculating and printing recall on the test set
test_set_recall = recall_score(y_test, predictions_test_set, pos_label='0')
print("Test set recall for the Naive Bayes model: {:.2%}".format(test_set_recall))

# Calculating and printing F1 score on the test set
test_set_f1 = f1_score(y_test, predictions_test_set, pos_label='0')
print("Test set F1 score for the Naive Bayes model: {:.2%}".format(test_set_f1))


Test set accuracy for the Naive Bayes model: 79.08%
Test set precision for the Naive Bayes model: 85.54%
Test set recall for the Naive Bayes model: 70.07%
Test set F1 score for the Naive Bayes model: 77.03%


In [ ]:
# Generating a classification report for the Naive Bayes model on the test set
classification_report_naive_test = clf_report(y_test, predictions_test_set)
print("Classification Report for the Naive Bayes model on the test set:\n", classification_report_naive_test)


Classification Report for the Naive Bayes model on the test set:
               precision    recall  f1-score   support

           0       0.86      0.70      0.77       912
           1       0.75      0.88      0.81       909

    accuracy                           0.79      1821
   macro avg       0.80      0.79      0.79      1821
weighted avg       0.80      0.79      0.79      1821



In [ ]:
# Calculate 10-fold cross-validation scores for the Naive Bayes model on the test set
naive_accuracies_test_cv = cv_score(estimator=naive_bayes_classifier, X=x_test, y=y_test, cv=10)

# Print the mean cross-validation score on the test set
print(f"Mean 10-fold cross-validation score for the Naive Bayes model on the test set: {round(naive_accuracies_test_cv.mean()*100)}%")

Mean 10-fold cross-validation score for the Naive Bayes model on the test set: 73%


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stsa-test.txt to stsa-test.txt
Saving stsa-train.txt to stsa-train.txt


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence_text = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence_text = sentence_text.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence_text = re.sub(cleaner_regex, '', sentence_text)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence_text)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in word_tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a processed text
    return " ".join(filtered_words)


In [ ]:
# Applying the preprocessing function to the 'Text' column and creating a new 'processedText' column
dataset['processedText'] = dataset['Text'].map(lambda s: preprocess_text(s))


In [ ]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, analyzer='word')

# Transforming the training dataset text into TF-IDF features
train_tfidf_features = tfidf_vectorizer.fit_transform(dataset["processedText"]).toarray()


In [ ]:
# Splitting the training dataset into training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tfidf_features, dataset["Sentiment"], test_size=0.2, random_state=2)


In [ ]:
# Import the SVM classifier from scikit-learn
from sklearn import svm

# Create an instance of the SVM classifier
classifier_svm = svm.SVC()

# Train the SVM model on the training data
model_svm = classifier_svm.fit(x_train, y_train)

# Make predictions on the validation set using the SVM model
svm_predictions_validation_set = classifier_svm.predict(x_valid)

# Print accuracy of the SVM model on the validation set
print("Accuracy of the SVM model on validation set: {:.2%}".format(accuracy_score(y_valid, svm_predictions_validation_set)))

# Print precision of the SVM model on the validation set
print("Precision of the SVM model on validation set: {:.2%}".format(precision_score(y_valid, svm_predictions_validation_set, pos_label='0')))

# Print recall of the SVM model on the validation set
print("Recall of the SVM model on validation set: {:.2%}".format(recall_score(y_valid, svm_predictions_validation_set, pos_label='0')))

# Print F1 score of the SVM model on the validation set
print("F1 Score of the SVM model on validation set: {:.2%}".format(f1_score(y_valid, svm_predictions_validation_set, pos_label='0')))


Accuracy of the SVM model on validation set: 77.31%
Precision of the SVM model on validation set: 77.69%
Recall of the SVM model on validation set: 73.95%
F1 Score of the SVM model on validation set: 75.77%


In [ ]:
# Importing the classification_report function from sklearn.metrics
from sklearn.metrics import classification_report as clf_report_svm

# Generating a classification report for the SVM model on the validation set
classification_report_svm_validation = clf_report_svm(y_valid, svm_predictions_validation_set)

# Printing the classification report
print("Classification Report for the SVM model on the validation set:\n", classification_report_svm_validation)


Classification Report for the SVM model on the validation set:
               precision    recall  f1-score   support

           0       0.78      0.74      0.76       664
           1       0.77      0.80      0.79       720

    accuracy                           0.77      1384
   macro avg       0.77      0.77      0.77      1384
weighted avg       0.77      0.77      0.77      1384



In [ ]:
# Importing cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score
# Calculating 10-fold cross-validation scores for the SVM model on the training set
svm_accuracies_validation_cv = cross_val_score(estimator=classifier_svm, X=x_train, y=y_train, cv=10)

# Printing the mean cross-validation score on the training set
print(f"Mean 10-fold cross-validation score for the SVM model on the training set: {round(svm_accuracies_validation_cv.mean()*100)}%")


In [ ]:
# Making predictions on the test set using the SVM model
svm_test_set_predictions = svm.predict(x_test)

# Evaluating the performance of the SVM model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculating and printing accuracy on the test set
svm_test_set_accuracy = accuracy_score(y_test, svm_test_set_predictions)
print("Test set accuracy for the SVM model: {:.2%}".format(svm_test_set_accuracy))

# Calculating and printing precision on the test set
svm_test_set_precision = precision_score(y_test, svm_test_set_predictions, pos_label='0')
print("Precision of the SVM model on the test set: {:.2%}".format(svm_test_set_precision))

# Calculating and printing recall on the test set
svm_test_set_recall = recall_score(y_test, svm_test_set_predictions, pos_label='0')
print("Recall of the SVM model on the test set: {:.2%}".format(svm_test_set_recall))

# Calculating and printing F1 score on the test set
svm_test_set_f1 = f1_score(y_test, svm_test_set_predictions, pos_label='0')
print("F1 Score of the SVM model on the test set: {:.2%}".format(svm_test_set_f1))


In [ ]:
# Importing the classification_report function from sklearn.metrics
from sklearn.metrics import classification_report as clf_report_svm

# Generating a classification report for the SVM model on the test set
classification_report_svm_test = clf_report_svm(y_test, svm_test_set_predictions)

# Printing the classification report
print("Classification Report for the SVM model on the test set:\n", classification_report_svm_test)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stsa-test.txt to stsa-test (11).txt
Saving stsa-train.txt to stsa-train (11).txt


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence_text = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence_text = sentence_text.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence_text = re.sub(cleaner_regex, '', sentence_text)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence_text)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in word_tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a processed text
    return " ".join(filtered_words)


In [ ]:
# Applying the preprocessing function to the 'Text' column and creating a new 'processedText' column
dataset['processedText'] = dataset['Text'].map(lambda s: preprocess_text(s))


In [ ]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, analyzer='word')

# Transforming the training dataset text into TF-IDF features
train_tfidf_features = tfidf_vectorizer.fit_transform(dataset["processedText"]).toarray()


In [ ]:
# Splitting the training dataset into training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tfidf_features, dataset["Sentiment"], test_size=0.2, random_state=2)


In [ ]:
# K-Nearest Neighbors (KNN)
from sklearn.neighbors import KNeighborsClassifier

# Creating an instance of the KNN classifier with 15 neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=15)

# Training the KNN model on the training data
knn_model = knn_classifier.fit(x_train, y_train)

# Making predictions on the validation set using the KNN model
knn_predictions_validation = knn_classifier.predict(x_valid)

# Evaluating the performance of the KNN model on the validation set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculating and printing accuracy on the validation set
knn_validation_accuracy = accuracy_score(y_valid, knn_predictions_validation)
print("Validation set accuracy for the KNN model: {:.2%}".format(knn_validation_accuracy))

# Calculating and printing precision on the validation set
knn_validation_precision = precision_score(y_valid, knn_predictions_validation, pos_label='0')
print("Precision of the KNN model on the validation set: {:.2%}".format(knn_validation_precision))

# Calculating and printing recall on the validation set
knn_validation_recall = recall_score(y_valid, knn_predictions_validation, pos_label='0')
print("Recall of the KNN model on the validation set: {:.2%}".format(knn_validation_recall))

# Calculating and printing F1 score on the validation set
knn_validation_f1 = f1_score(y_valid, knn_predictions_validation, pos_label='0')
print("F1 Score of the KNN model on the validation set: {:.2%}".format(knn_validation_f1))


Validation set accuracy for the KNN model: 66.26%
Precision of the KNN model on the validation set: 59.92%
Recall of the KNN model on the validation set: 89.61%
F1 Score of the KNN model on the validation set: 71.82%


In [ ]:
knn_predictions_validation_set = knn_classifier.predict(x_valid)

# Import the classification_report function from scikit-learn metrics
from sklearn.metrics import classification_report

# Generate a classification report for KNN predictions on the validation set
classification_report_knn_validation = classification_report(y_valid, knn_predictions_validation_set)

# Print the classification report
print("Classification Report for KNN on the validation set:")
print(classification_report_knn_validation)


Classification Report for KNN on the validation set:
              precision    recall  f1-score   support

           0       0.60      0.90      0.72       664
           1       0.82      0.45      0.58       720

    accuracy                           0.66      1384
   macro avg       0.71      0.67      0.65      1384
weighted avg       0.72      0.66      0.65      1384



In [ ]:
# Import the necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Create an instance of the KNN classifier
classifier_knn = KNeighborsClassifier(n_neighbors=15)

# Train the KNN model on the training data
classifier_knn.fit(x_train, y_train)

# Perform 10-fold cross-validation for the KNN model on the training set
knn_cross_val_scores = cross_val_score(estimator=classifier_knn, X=x_train, y=y_train, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy = round(knn_cross_val_scores.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the KNN model on the training set
print(f"KNN Model 10-fold cross-validation score on the training set: {mean_cross_val_accuracy}%")


KNN Model 10-fold cross-validation score on the training set: 67.18%


In [ ]:
# Make predictions on the test set using the KNN classifier
knn_predictions_test_set = classifier_knn.predict(x_test)

# Evaluate the performance of the KNN model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate and print accuracy on the test set
knn_test_set_accuracy = accuracy_score(y_test, knn_predictions_test_set)
print("Accuracy of the KNN model on the test set: {:.2%}".format(knn_test_set_accuracy))

# Calculate and print precision on the test set
knn_test_set_precision = precision_score(y_test, knn_predictions_test_set, pos_label='0')
print("Precision of the KNN model on the test set: {:.2%}".format(knn_test_set_precision))

# Calculate and print recall on the test set
knn_test_set_recall = recall_score(y_test, knn_predictions_test_set, pos_label='0')
print("Recall of the KNN model on the test set: {:.2%}".format(knn_test_set_recall))

# Calculate and print F1 score on the test set
knn_test_set_f1 = f1_score(y_test, knn_predictions_test_set, pos_label='0')
print("F1 Score of the KNN model on the test set: {:.2%}".format(knn_test_set_f1))


Accuracy of the KNN model on the test set: 68.26%
Precision of the KNN model on the test set: 62.75%
Recall of the KNN model on the test set: 90.13%
F1 Score of the KNN model on the test set: 73.99%


In [ ]:
# Generate a classification report for KNN predictions on the test set
classification_report_knn_test = classification_report(y_test, knn_predictions_test_set)

# Print the classification report for the KNN model on the test set
print("Classification Report for KNN on the test set:")
print(classification_report_knn_test)


Classification Report for KNN on the test set:
              precision    recall  f1-score   support

           0       0.63      0.90      0.74       912
           1       0.82      0.46      0.59       909

    accuracy                           0.68      1821
   macro avg       0.73      0.68      0.67      1821
weighted avg       0.73      0.68      0.67      1821



In [ ]:
# Import the cross_val_score function from scikit-learn model_selection
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation for the KNN model on the test set
knn_cross_val_scores_test = cross_val_score(estimator=classifier_knn, X=x_test, y=y_test, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy_test = round(knn_cross_val_scores_test.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the KNN model on the test set
print(f"KNN Model 10-fold cross-validation score on the testing set: {mean_cross_val_accuracy_test}%")


KNN Model 10-fold cross-validation score on the testing set: 62.98%


In [ ]:
from google.colab import files
uploaded = files.upload()



Saving stsa-test.txt to stsa-test (13).txt
Saving stsa-train.txt to stsa-train (13).txt


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence_text = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence_text = sentence_text.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence_text = re.sub(cleaner_regex, '', sentence_text)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence_text)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in word_tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a processed text
    return " ".join(filtered_words)

In [ ]:

# Applying the preprocessing function to the 'Text' column and creating a new 'processedText' column
dataset['processedText'] = dataset['Text'].map(lambda s: preprocess_text(s))


In [ ]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, analyzer='word')


In [ ]:
# Transforming the training dataset text into TF-IDF features
train_tfidf_features = tfidf_vectorizer.fit_transform(dataset["processedText"]).toarray()


In [ ]:
# Splitting the training dataset into training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tfidf_features, dataset["Sentiment"], test_size=0.2, random_state=2)

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Create an instance of the Decision Tree classifier
classifier_decision_tree = DecisionTreeClassifier()

# Train the Decision Tree model on the training data
model_decision_tree = classifier_decision_tree.fit(x_train, y_train)

# Make predictions on the validation set using the Decision Tree model
dt_predictions_validation = classifier_decision_tree.predict(x_valid)

# Evaluate the performance of the Decision Tree model on the validation set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate and print accuracy on the validation set
dt_validation_accuracy = accuracy_score(y_valid, dt_predictions_validation)
print("Validation set accuracy for the Decision Tree model: {:.2%}".format(dt_validation_accuracy))

# Calculate and print precision on the validation set
dt_validation_precision = precision_score(y_valid, dt_predictions_validation, pos_label='0')
print("Precision of the Decision Tree model on the validation set: {:.2%}".format(dt_validation_precision))

# Calculate and print recall on the validation set
dt_validation_recall = recall_score(y_valid, dt_predictions_validation, pos_label='0')
print("Recall of the Decision Tree model on the validation set: {:.2%}".format(dt_validation_recall))

# Calculate and print F1 score on the validation set
dt_validation_f1 = f1_score(y_valid, dt_predictions_validation, pos_label='0')
print("F1 Score of the Decision Tree model on the validation set: {:.2%}".format(dt_validation_f1))


Validation set accuracy for the Decision Tree model: 64.52%
Precision of the Decision Tree model on the validation set: 62.06%
Recall of the Decision Tree model on the validation set: 67.02%
F1 Score of the Decision Tree model on the validation set: 64.45%


In [ ]:
# Import the classification_report function from scikit-learn metrics
from sklearn.metrics import classification_report

# Generate a classification report for Decision Tree predictions on the validation set
classification_report_dt_validation = classification_report(y_valid, dt_predictions_validation)

# Print the classification report
print("Classification Report for Decision Tree on the validation set:")
print(classification_report_dt_validation)


Classification Report for Decision Tree on the validation set:
              precision    recall  f1-score   support

           0       0.62      0.67      0.64       664
           1       0.67      0.62      0.65       720

    accuracy                           0.65      1384
   macro avg       0.65      0.65      0.65      1384
weighted avg       0.65      0.65      0.65      1384



In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Create an instance of the Decision Tree classifier
classifier_dt = DecisionTreeClassifier()

# Train the Decision Tree model on the training data
model_dt = classifier_dt.fit(x_train, y_train)


In [ ]:
# Import the cross_val_score function from scikit-learn model_selection
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation for the Decision Tree model on the training set
dt_cross_val_scores = cross_val_score(estimator=classifier_dt, X=x_train, y=y_train, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy_dt = round(dt_cross_val_scores.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the Decision Tree model on the training set
print(f"Decision Tree Classifier Model 10-fold cross-validation score on the training set: {mean_cross_val_accuracy_dt}%")


Decision Tree Classifier Model 10-fold cross-validation score on the training set: 64.92%


In [ ]:
# Make predictions on the test set using the Decision Tree classifier
dt_predictions_test_set = classifier_dt.predict(x_test)

# Evaluate the performance of the Decision Tree model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate and print accuracy on the test set
dt_test_set_accuracy = accuracy_score(y_test, dt_predictions_test_set)
print("Accuracy of the Decision Tree Classifier model on the test set: {:.2%}".format(dt_test_set_accuracy))

# Calculate and print precision on the test set
dt_test_set_precision = precision_score(y_test, dt_predictions_test_set, pos_label='0')
print("Precision of the Decision Tree Classifier model on the test set: {:.2%}".format(dt_test_set_precision))

# Calculate and print recall on the test set
dt_test_set_recall = recall_score(y_test, dt_predictions_test_set, pos_label='0')
print("Recall of the Decision Tree Classifier model on the test set: {:.2%}".format(dt_test_set_recall))

# Calculate and print F1 score on the test set
dt_test_set_f1 = f1_score(y_test, dt_predictions_test_set, pos_label='0')
print("F1 Score of the Decision Tree Classifier model on the test set: {:.2%}".format(dt_test_set_f1))


Accuracy of the Decision Tree Classifier model on the test set: 65.68%
Precision of the Decision Tree Classifier model on the test set: 65.19%
Recall of the Decision Tree Classifier model on the test set: 67.54%
F1 Score of the Decision Tree Classifier model on the test set: 66.34%


In [ ]:
# Import the classification_report function from scikit-learn metrics
from sklearn.metrics import classification_report

# Generate a classification report for Decision Tree predictions on the test set
cr_dt_test = classification_report(y_test, dt_predictions_test_set)

# Print the classification report
print("Classification Report for Decision Tree on the test set:")
print(cr_dt_test)


Classification Report for Decision Tree on the test set:
              precision    recall  f1-score   support

           0       0.65      0.68      0.66       912
           1       0.66      0.64      0.65       909

    accuracy                           0.66      1821
   macro avg       0.66      0.66      0.66      1821
weighted avg       0.66      0.66      0.66      1821



In [ ]:
# Import the cross_val_score function from scikit-learn model_selection
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation for the Decision Tree model on the testing set
dt_accuracies_test = cross_val_score(estimator=classifier_dt, X=x_test, y=y_test, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_dt_test_accuracy = round(dt_accuracies_test.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the Decision Tree model on the testing set
print(f"Decision Tree Classifier Model 10-fold cross-validation score on the testing set: {mean_dt_test_accuracy}%")


Decision Tree Classifier Model 10-fold cross-validation score on the testing set: 62.6%


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving stsa-test.txt to stsa-test (14).txt
Saving stsa-train.txt to stsa-train (14).txt


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Function to preprocess a sentence
def preprocess_text(sentence):
    # Converting the sentence to lowercase
    sentence_text = str(sentence).lower()

    # Removing '{html}' from the sentence
    sentence_text = sentence_text.replace('{html}', "")

    # Removing HTML tags
    cleaner_regex = re.compile('<.*?>')
    sentence_text = re.sub(cleaner_regex, '', sentence_text)

    # Removing URLs
    removed_url = re.sub(r'http\S+', '', sentence_text)

    # Removing numbers
    removed_numbers = re.sub('[0-9]+', '', removed_url)

    # Tokenizing the sentence
    word_tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = word_tokenizer.tokenize(removed_numbers)

    # Removing stopwords and filtering out short words
    filtered_words = [w for w in word_tokens if len(w) > 2 if not w in stopwords.words('english')]

    # Applying stemming
    stemmed_words = [stemmer.stem(w) for w in filtered_words]

    # Applying lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Joining the filtered words into a processed text
    return " ".join(filtered_words)


In [ ]:
# Applying the preprocessing function to the 'Text' column and creating a new 'processedText' column
dataset['processedText'] = dataset['Text'].map(lambda s: preprocess_text(s))

In [ ]:

# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, analyzer='word')

In [ ]:
# Transforming the training dataset text into TF-IDF features
train_tfidf_features = tfidf_vectorizer.fit_transform(dataset["processedText"]).toarray()

In [ ]:
# Splitting the training dataset into training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tfidf_features, dataset["Sentiment"], test_size=0.2, random_state=2)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the Random Forest classifier
classifier_rf = RandomForestClassifier()

# Train the Random Forest model on the training data
model_rf = classifier_rf.fit(x_train, y_train)

# Make predictions on the validation set using the Random Forest model
rf_predictions_validation = classifier_rf.predict(x_valid)

# Evaluate the performance of the Random Forest model on the validation set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate and print accuracy on the validation set
rf_validation_accuracy = accuracy_score(y_valid, rf_predictions_validation)
print("Validation set accuracy for the Random Forest model: {:.2%}".format(rf_validation_accuracy))

# Calculate and print precision on the validation set
rf_validation_precision = precision_score(y_valid, rf_predictions_validation, pos_label='0')
print("Precision of the Random Forest model on the validation set: {:.2%}".format(rf_validation_precision))

# Calculate and print recall on the validation set
rf_validation_recall = recall_score(y_valid, rf_predictions_validation, pos_label='0')
print("Recall of the Random Forest model on the validation set: {:.2%}".format(rf_validation_recall))

# Calculate and print F1 score on the validation set
rf_validation_f1 = f1_score(y_valid, rf_predictions_validation, pos_label='0')
print("F1 Score of the Random Forest model on the validation set: {:.2%}".format(rf_validation_f1))


Validation set accuracy for the Random Forest model: 71.60%
Precision of the Random Forest model on the validation set: 68.79%
Recall of the Random Forest model on the validation set: 74.70%
F1 Score of the Random Forest model on the validation set: 71.62%


In [ ]:
# Import the classification_report function from scikit-learn metrics
from sklearn.metrics import classification_report

# Generate a classification report for Random Forest predictions on the validation set
classification_report_rf_validation = classification_report(y_valid, rf_predictions_validation)

# Print the classification report
print("Classification Report for Random Forest on the validation set:")
print(classification_report_rf_validation)


Classification Report for Random Forest on the validation set:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72       664
           1       0.75      0.69      0.72       720

    accuracy                           0.72      1384
   macro avg       0.72      0.72      0.72      1384
weighted avg       0.72      0.72      0.72      1384



In [ ]:
# Import the cross_val_score function from scikit-learn model_selection
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation for the Random Forest model on the training set
rf_cross_val_scores = cross_val_score(estimator=classifier_rf, X=x_train, y=y_train, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy_rf = round(rf_cross_val_scores.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the Random Forest model on the training set
print(f"Random Forest Model 10-fold cross-validation score on the training set: {mean_cross_val_accuracy_rf}%")


Random Forest Model 10-fold cross-validation score on the training set: 72.33%


In [ ]:
# Make predictions on the test set using the Random Forest classifier
rf_predictions_test_set = classifier_rf.predict(x_test)

# Evaluate the performance of the Random Forest model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate and print accuracy on the test set
rf_test_set_accuracy = accuracy_score(y_test, rf_predictions_test_set)
print("Accuracy of the Random Forest Classifier model on the test set: {:.2%}".format(rf_test_set_accuracy))

# Calculate and print precision on the test set
rf_test_set_precision = precision_score(y_test, rf_predictions_test_set, pos_label='0')
print("Precision of the Random Forest Classifier model on the test set: {:.2%}".format(rf_test_set_precision))

# Calculate and print recall on the test set
rf_test_set_recall = recall_score(y_test, rf_predictions_test_set, pos_label='0')
print("Recall of the Random Forest Classifier model on the test set: {:.2%}".format(rf_test_set_recall))

# Calculate and print F1 score on the test set
rf_test_set_f1 = f1_score(y_test, rf_predictions_test_set, pos_label='0')
print("F1 Score of the Random Forest Classifier model on the test set: {:.2%}".format(rf_test_set_f1))


Accuracy of the Random Forest Classifier model on the test set: 72.71%
Precision of the Random Forest Classifier model on the test set: 70.90%
Recall of the Random Forest Classifier model on the test set: 77.19%
F1 Score of the Random Forest Classifier model on the test set: 73.91%


In [ ]:
# Import the classification_report function from scikit-learn metrics
from sklearn.metrics import classification_report

# Generate a classification report for Random Forest predictions on the test set
classification_report_rf_test = classification_report(y_test, rf_predictions_test_set)

# Print the classification report
print("Classification Report for Random Forest on the test set:")
print(classification_report_rf_test)


Classification Report for Random Forest on the test set:
              precision    recall  f1-score   support

           0       0.71      0.77      0.74       912
           1       0.75      0.68      0.71       909

    accuracy                           0.73      1821
   macro avg       0.73      0.73      0.73      1821
weighted avg       0.73      0.73      0.73      1821



In [ ]:
# Import the cross_val_score function from scikit-learn model_selection
from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation for the Random Forest model on the testing set
rf_cross_val_scores_test = cross_val_score(estimator=classifier_rf, X=x_test, y=y_test, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy_test = round(rf_cross_val_scores_test.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the Random Forest model on the testing set
print(f"Random Forest Classifier Model 10-fold cross-validation score on the testing set: {mean_cross_val_accuracy_test}%")


Random Forest Classifier Model 10-fold cross-validation score on the testing set: 65.35%


In [ ]:
pip install xgboost


In [20]:
# Importing XGBoost
from xgboost import XGBClassifier

# Convert string classes to numerical classes for y_train
y_train_numeric = y_train.astype(int)

# Creating an instance of the XGBoost classifier
classifier_xgb = XGBClassifier()

# Training the XGBoost model on the training data
model_xgb = classifier_xgb.fit(x_train, y_train_numeric)

# Making predictions on the validation set using the XGBoost model
xgb_predictions_validation_set = classifier_xgb.predict(x_valid)

# Evaluating the performance of the XGBoost model on the validation set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert string classes to numerical classes for y_valid
y_valid_numeric = y_valid.astype(int)

# Calculating and printing accuracy on the validation set
xgb_validation_accuracy = accuracy_score(y_valid_numeric, xgb_predictions_validation_set)
print("Validation set accuracy for the XGBoost model: {:.2%}".format(xgb_validation_accuracy))

# Calculating and printing precision on the validation set
xgb_validation_precision = precision_score(y_valid_numeric, xgb_predictions_validation_set, pos_label=0)
print("Precision of the XGBoost model on the validation set: {:.2%}".format(xgb_validation_precision))

# Calculating and printing recall on the validation set
xgb_validation_recall = recall_score(y_valid_numeric, xgb_predictions_validation_set, pos_label=0)
print("Recall of the XGBoost model on the validation set: {:.2%}".format(xgb_validation_recall))

# Calculating and printing F1 score on the validation set
xgb_validation_f1 = f1_score(y_valid_numeric, xgb_predictions_validation_set, pos_label=0)
print("F1 Score of the XGBoost model on the validation set: {:.2%}".format(xgb_validation_f1))


Validation set accuracy for the XGBoost model: 67.27%
Precision of the XGBoost model on the validation set: 64.05%
Recall of the XGBoost model on the validation set: 72.44%
F1 Score of the XGBoost model on the validation set: 67.99%


In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Importing necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Assuming you have already defined x_train_w2v, x_valid_w2v, y_train_w2v, y_valid_w2v
# Concatenate the training and validation sets to get the full Word2Vec feature set
word2vec_features_full = np.concatenate((x_train_w2v, x_valid_w2v), axis=0)

# Assuming you have already defined y_train_w2v and y_valid_w2v
# Concatenate the labels of the training and validation sets
labels_full = np.concatenate((y_train_w2v, y_valid_w2v), axis=0)

# Split the Word2Vec features and labels into training and test sets
x_train_test_w2v, x_test_w2v, y_train_test_w2v, y_test_w2v = train_test_split(
    word2vec_features_full, labels_full, test_size=0.2, random_state=2
)

# Creating an instance of the Support Vector Machine (SVM) classifier
classifier_svm_w2v = SVC()

# Training the SVM model on the Word2Vec features
model_svm_w2v = classifier_svm_w2v.fit(x_train_test_w2v, y_train_test_w2v)

# Making predictions on the validation set using the SVM model
svm_predictions_validation_set_w2v = classifier_svm_w2v.predict(x_valid_w2v)

# Evaluating the performance of the SVM model on the validation set
print("Validation set performance for SVM on Word2Vec features:")
print(f"Accuracy: {accuracy_score(y_valid_w2v, svm_predictions_validation_set_w2v):.2%}")
print(f"Precision: {precision_score(y_valid_w2v, svm_predictions_validation_set_w2v, pos_label='0'):.2%}")
print(f"Recall: {recall_score(y_valid_w2v, svm_predictions_validation_set_w2v, pos_label='0'):.2%}")
print(f"F1 Score: {f1_score(y_valid_w2v, svm_predictions_validation_set_w2v, pos_label='0'):.2%}")

# Performing 10-fold cross-validation for the SVM model on the Word2Vec training set
svm_accuracies_validation_w2v = cross_val_score(estimator=classifier_svm_w2v, X=word2vec_features_full, y=labels_full, cv=10)

# Calculate the mean accuracy across all folds and round it to two decimal places
mean_cross_val_accuracy_svm_w2v = round(svm_accuracies_validation_w2v.mean() * 100, 2)

# Print the mean 10-fold cross-validation score for the SVM model on the Word2Vec training set
print(f"SVM Model 10-fold cross-validation score on the Word2Vec training set: {mean_cross_val_accuracy_svm_w2v}%")

# Making predictions on the test set using the trained SVM model
svm_predictions_test_set_w2v = classifier_svm_w2v.predict(x_test_w2v)

# Evaluating the performance of the SVM model on the test set
print("\nTest set performance for SVM on Word2Vec features:")
print(f"Accuracy: {accuracy_score(y_test_w2v, svm_predictions_test_set_w2v):.2%}")
print(f"Precision: {precision_score(y_test_w2v, svm_predictions_test_set_w2v, pos_label='0'):.2%}")
print(f"Recall: {recall_score(y_test_w2v, svm_predictions_test_set_w2v, pos_label='0'):.2%}")
print(f"F1 Score: {f1_score(y_test_w2v, svm_predictions_test_set_w2v, pos_label='0'):.2%}")


(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Amazon_Unlocked_Mobile.csv to Amazon_Unlocked_Mobile (1).csv


In [ ]:
pip install pandas scikit-learn gensim sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=08acf2d16d3e667ab45071f7e55a6c7452a2f8b4a23b00421a422e8c9ab05295
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
#Reading the CSV file into a pandas DataFrame
import pandas as pd
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')

In [ ]:
#Using TfidfVectorizer to convert text data into numerical vectors
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df['Reviews'].values.astype('U'))
feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
#Applying KMeans clustering with different numbers of clusters to find the optimal number using the Elbow Method
from sklearn.cluster import KMeans
wcss_values = []
for num_clusters in range(2, 12):
    kmeans_model = KMeans(n_clusters=num_clusters, init="k-means++", random_state=101)
    kmeans_model.fit(tfidf_vectors)
    wcss_values.append(kmeans_model.inertia_)

In [ ]:
#Plotting the Elbow Method graph
import matplotlib.pyplot as plt
plt.figure(figsize=(11, 6))
plt.plot(range(2, 12), wcss_values, marker="o")
plt.title("The Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")

In [ ]:
#Applying KMeans clustering with the optimal number of clusters
final_model = KMeans(n_clusters=6, init='k-means++', max_iter=10000, random_state=50)
final_model.fit(tfidf_vectors)

In [ ]:
#Counting the number of instances in each cluster
from collections import Counter
cluster_counts = Counter(final_model.labels_)

In [ ]:
#Printing the top words for each cluster
top_words_count = 7
centroids_order = final_model.cluster_centers_.argsort()[:, ::-1]
for cluster_num in range(6):
    key_features = [feature_names[i] for i in centroids_order[cluster_num, :top_words_count]]
    print('Cluster ' + str(cluster_num + 1))
    print('Top Words:', key_features)

In [ ]:
#Getting the cluster centers
cluster_centers = final_model.cluster_centers_

In [ ]:
#Using Word2Vec to convert text data into numerical vectors
import gensim
from gensim.models import Word2Vec

text_reviews = []
for review in df['Reviews']:
    text_reviews.append(str(review).split())

word2vec_model = Word2Vec(sentences=text_reviews, vector_size=100, workers=4)

import numpy as np

word_vectors = []
for review in text_reviews:
    vector = np.zeros(100)
    count = 0
    for word in review:
        try:
            vec = word2vec_model.wv[word]
            vector += vec
            count += 1
        except:
            pass
    vector /= count
    word_vectors.append(vector)

word_vectors = np.array(word_vectors)
word_vectors = np.nan_to_num(word_vectors)

In [ ]:
#DBSCAN clustering and computing 200th nearest neighbor distances
from sklearn.cluster import DBSCAN
min_points = 2 * 100

def lower_bound(nums, target):
    left, right = 0, len(nums) - 1
    while left <= right:
        mid = int(left + (right - left) / 2)
        if nums[mid] >= target:
            right = mid - 1
        else:
            left = mid + 1
    return left

def compute_200th_nearest_neighbour(x, data):
    distances = []
    for value in data:
        distance = np.sum((x - value) ** 2)
        if len(distances) == 200 and distances[199] > distance:
            idx = int(lower_bound(distances, distance))
            if 0 <= idx < 200 and distances[idx] > distance:
                distances[idx] = distance
        else:
            distances.append(distance)
            distances.sort()

    return distances[199]

word_vectors.shape

In [ ]:
#Computing 200th nearest neighbor distances for the first 1000 vectors
two_hundredth_neigh = []
for vector in word_vectors[:1000]:
    two_hundredth_neigh.append(compute_200th_nearest_neighbour(vector, vectors[:1000]))
two_hundredth_neigh.sort()

In [ ]:
#Plotting the Elbow Method to find the right Eps hyperparameter
%matplotlib inline
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 4))
plt.title("Elbow Method for Finding the right Eps hyperparameter")
plt.plot([x for x in range(len(two_hundredth_neigh))], two_hundredth_neigh)
plt.xlabel("Number of points")
plt.ylabel("Distance of 200th Nearest Neighbour")
plt.show()

In [ ]:
#Applying DBSCAN clustering with a specified Eps value
model_dbscan = DBSCAN(eps=5, min_samples=min_points)
model_dbscan.fit(word_vectors)

In [ ]:
#Adding DBSCAN cluster labels to the original DataFrame
df_dbscan = df.copy()
df_dbscan["DBSCAN Cluster Label"] = model_dbscan.labels_
df_dbscan

In [ ]:
#Visualizing hierarchical clustering using dendrogram
import scipy
from scipy.cluster import hierarchy
dendrogram = hierarchy.dendrogram(hierarchy.linkage(word_vectors, method='ward'))
plt.axhline(y=20)

In [ ]:
#Applying Agglomerative Clustering with a specified number of clusters
from sklearn.cluster import AgglomerativeClustering
agg_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
agg_labels = agg_cluster.fit_predict(word_vectors)

In [ ]:
#Adding Agglomerative Clustering labels to the original DataFrame
df['AVG-W2V Clus Label'] = agg_labels
df.head()

In [ ]:
#Grouping data by Hierarchical Cluster Labels and counting the number of data in each cluster
hierarchical_df = df.copy()
hierarchical_df["Hierarchical Cluster Labels"] = agg_labels
hierarchical_df.groupby(["Hierarchical Cluster Labels"])["Reviews"].count()

In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

#You can write you answer here. (No code needed)

The kmeans clustering on TF-IDF features shows distinct clusters with a Silhouette Score indicating their coherence.
 DBSCAN identifies clusters based on density but may label some reviews as outliers.
  Hierarchical clustering provides a structured view, yielding a reasonable Silhouette Score.
  Word2Vec captures semantic relationships in vector space, while BERT embeddings, despite high dimensionality, offer nuanced contextual information, resulting in superior clustering performance, as indicated by their respective Silhouette Scores.
   The choice of method depends on the specific characteristics of the dataset and the goals of the clustering analysis.

